In [1]:
import os
import sys
base_dir = os.path.split(os.getcwd())[0]
sys.path.append(base_dir)
sys.path.append(os.path.join(base_dir, "APIClient"))

In [3]:
from APIClient.company_handler import CompanyHandler

company_handler = CompanyHandler(
    path_companies_to_extract="D:/PythonProjects/MasterThesisResearch/APIClient/data/nasdaq/energy_all.csv",
)

list_of_companies_base = []
for company in company_handler.load_chosen_companies_from_db():
    list_of_companies_base.append(company)

Missing data in DB for ticker  AMR
Missing data in DB for ticker  ARCH
Missing data in DB for ticker  BKEP
Missing data in DB for ticker  BORR
Missing data in DB for ticker  BP
Missing data in DB for ticker  BROG
Missing data in DB for ticker  BROGW
Missing data in DB for ticker  CEQP
Missing data in DB for ticker  CHK
Missing data in DB for ticker  CHKEL
Missing data in DB for ticker  CHKEW
Missing data in DB for ticker  CRC
Missing data in DB for ticker  E
Missing data in DB for ticker  EC
Missing data in DB for ticker  ENB
Missing data in DB for ticker  ENBA
Missing data in DB for ticker  EQNR
Missing data in DB for ticker  FLMN
Missing data in DB for ticker  GLNG
Missing data in DB for ticker  GLOP
Missing data in DB for ticker  GLP
Missing data in DB for ticker  HPK
Missing data in DB for ticker  INDO
Missing data in DB for ticker  METC
Missing data in DB for ticker  NVVE
Missing data in DB for ticker  OXY
Missing data in DB for ticker  PBA
Missing data in DB for ticker  PBR
Missi

In [4]:
# Get all companies from REST API
import requests
import json

list_of_companies = []
list_of_years = range(2017, 2023)
for company in list_of_companies_base:
    list_of_curr_company = []
    for year in list_of_years:
    
        resp = json.loads(requests.get(f"http://localhost:8000/api/v1/company/{company['cik']}?year={year}").text)
    
        if resp['code'] == 200:
            if len(resp['data']) > 1:
                print(f"More entries for {company['cik']}, year {year}")
            else:
                list_of_curr_company.append(resp['data'][0])
            
    list_of_companies.append(list_of_curr_company)
    

In [6]:
# Save list of companies in json for easy access

import json

with open('data/list_of_companies.json', 'w') as f:
    json.dump(list_of_companies, f)

### Read the json if you have it saved for energy companies

In [223]:
# Read file 

import json

with open('data/list_of_companies.json', 'r') as f:
    list_of_companies= json.load(f)

In [198]:
from collections import defaultdict

list_of_results = []
for list_curr_company in list_of_companies:
    company_results = {}
    for curr_dict in list_curr_company:
        q_to_metadata = defaultdict(list)
        for quarter in curr_dict['quarters']:
            for metadata_dict in quarter['metadata']:
                if metadata_dict['risk_section']:
                    metadata_dict['risk_section_len'] = len(metadata_dict['risk_section'].split())
                else:
                    metadata_dict['risk_section_len'] = 0
                    
                if metadata_dict['mda_section']:
                    metadata_dict['mda_section_len'] = len(metadata_dict['mda_section'].split())
                else:
                    metadata_dict['mda_section_len'] = 0
                    
                if metadata_dict['qqd_section']:
                    metadata_dict['qqd_section_len'] = len(metadata_dict['qqd_section'].split())
                else:
                    metadata_dict['qqd_section_len'] = 0
                
                copy_metadata_dict = metadata_dict.copy()
                del copy_metadata_dict['qqd_section']
                del copy_metadata_dict['risk_section']
                del copy_metadata_dict['mda_section']
                q_to_metadata[quarter['q']].append(copy_metadata_dict)
            
        company_results['cik'] = curr_dict['cik']
        company_results[curr_dict['year']] = q_to_metadata
    
    list_of_results.append(company_results)

## Analysis on the data

In [199]:
print(f"Number of companies is {len(list_of_results)}")

Number of companies is 172


In [200]:
import plotly.express as px
from collections import defaultdict
import pandas as pd

def get_df_plotly(use_data='all'): # possible type: 'all', 'year'
    if use_data == 'all':
        list_for_df = []
        for company in list_of_results:
            for year in range(2017, 2022):
                for q in range(1,5):
                    if not company.get(year,None):
                        continue
                    curr_q = company[year].get(q, None)
                    
                    if not curr_q:
                        continue
                    for dict_metadata in curr_q:
                        if dict_metadata['risk_section_len'] > 0:
                            list_for_df.append({'section_type': 'risk', 'len_text': dict_metadata['risk_section_len'],
                                                'cik':company['cik'], 'year':year, 'q': q, 'filing_type': dict_metadata['type']})
                        if dict_metadata['mda_section_len'] > 0:
                            list_for_df.append({'section_type': 'mda', 'len_text': dict_metadata['mda_section_len'],
                                                'cik':company['cik'], 'year':year, 'q': q, 'filing_type': dict_metadata['type']})
                        if dict_metadata['qqd_section_len'] > 0:
                            list_for_df.append({'section_type': 'qqd', 'len_text': dict_metadata['qqd_section_len'],
                                                'cik':company['cik'], 'year':year, 'q': q, 'filing_type': dict_metadata['type']})
        
    return pd.DataFrame(list_for_df)
    

In [201]:
df = get_df_plotly()

In [181]:
fig = px.histogram(df, x="len_text",color="section_type", marginal="rug",
                   hover_data=df.columns, title="All filings from 2017 to 2022 by section type")
fig.show()

In [182]:
fig = px.histogram(df, x="len_text",color="filing_type", marginal="rug",
                   hover_data=df.columns, title="All filings from 2017 to 2022 by type of filing")
fig.show()

In [183]:
fig = px.histogram(df.loc[df['section_type'] == 'risk'], x="len_text", color="filing_type", marginal="rug",
                   hover_data=df.columns, title="Risk sections from 2017 to 2022 by filing type")
fig.show()

## Criterias to use company in the data
- If data is not missing for a sequence of 3 or more for risk and mda.

In [202]:
# Leave only 10K, 10Q, 10KA, 10QA
df_cleaned = df.loc[df['filing_type'].isin(['10K', '10Q', '10KA', '10QA'])]

In [203]:
df_cleaned.loc[(df_cleaned['cik'] == 70145) & (df_cleaned['section_type'] == "mda")]

,section_type,len_text,cik,year,q,filing_type
3821,mda,10270,70145,2017,1,10Q
3823,mda,12386,70145,2017,2,10Q
3825,mda,12750,70145,2017,3,10Q
3827,mda,19668,70145,2017,4,10K
3830,mda,10992,70145,2018,1,10Q
3832,mda,13064,70145,2018,2,10Q
3834,mda,13765,70145,2018,3,10Q
3836,mda,21162,70145,2018,4,10K
3839,mda,11482,70145,2019,1,10Q
3841,mda,12868,70145,2019,2,10Q


In [204]:
# Create new dfs based on section type
df_risk_only = df_cleaned.loc[df_cleaned['section_type'] == "risk"]
df_mda_only = df_cleaned.loc[df_cleaned['section_type'] == "mda"]

In [205]:
from collections import defaultdict

def get_missing_data_companies_df(df):
    dict_results = defaultdict(list)
    list_ciks = list(set(df['cik']))
    for cik in list_ciks:
        cik_df = df.loc[df['cik'] == cik]
        for year in range(2017,2022):
            year_df = cik_df.loc[cik_df['year'] == year]
            q_series = year_df['q'].values
            for q in range(1,5):
                if q not in q_series:
                    dict_results[cik].append(f"{year}_{q}")
                    
    return dict_results

In [206]:
missing_mda = get_missing_data_companies_df(df_mda_only)
missing_risk = get_missing_data_companies_df(df_risk_only)

In [207]:
print(f"MDA df number of companies: {len(set(df_mda_only['cik']))}")
print(f"MDA number of companies with missing data: {len(missing_mda)}")
print(f"RISK df number of companies: {len(set(df_risk_only['cik']))}")
print(f"RISK number of companies with missing data: {len(missing_risk)}")

MDA df number of companies: 169
MDA number of companies with missing data: 61
RISK df number of companies: 169
RISK number of companies with missing data: 78


In [208]:
def get_companies_to_remove(missing_dict_section, consecutive_count=3):
    # remove companies that have consecutive missing filings e.g 2017_1, 2017_2, 2017_3
    dict_of_cik_consecutives = {}
    for cik, list_year_q in missing_dict_section.items():
        curr_consecutive_count = 1
        best_consecutive_count = 1
        for i in range(len(list_year_q) - 1):
            curr_year, curr_q = [int(x) for x in list_year_q[i].split('_')]
            next_year, next_q = [int(x) for x in list_year_q[i+1].split('_')]
            
            
            if curr_q + 1 == next_q and curr_q < 4 and curr_year == next_year:
                curr_consecutive_count += 1
                if curr_consecutive_count > best_consecutive_count:
                    best_consecutive_count = curr_consecutive_count
            elif curr_q == 4 and curr_year + 1 == next_year and next_q == 1:
                curr_consecutive_count += 1
                if curr_consecutive_count > best_consecutive_count:
                    best_consecutive_count = curr_consecutive_count
            else:
                if curr_consecutive_count > best_consecutive_count:
                    best_consecutive_count = curr_consecutive_count
                curr_consecutive_count = 1
        
        if best_consecutive_count >= consecutive_count:
            dict_of_cik_consecutives[cik] = best_consecutive_count
            
    return dict_of_cik_consecutives

In [209]:
# Check duplicated ciks in the two dicts for statistics and remove all from original DF

dict_mda_consecutives = get_companies_to_remove(missing_mda, 3)
dict_risk_consecutives = get_companies_to_remove(missing_risk, 3)

In [210]:
count_duplicated = 0
for cik, count_consecutives in dict_mda_consecutives.items():
    if cik in dict_risk_consecutives:
        count_duplicated += 1
        print(f"Data for CIK {cik} is missing from both MDA and Risk")

print(f"{count_duplicated/len(dict_mda_consecutives) * 100} % of MDA are also in Risk")       
print(f"Len of mda ciks {len(dict_mda_consecutives)}")
print(f"Len of risk ciks {len(dict_risk_consecutives)}")

Data for CIK 1841666 is missing from both MDA and Risk
Data for CIK 1423902 is missing from both MDA and Risk
Data for CIK 1724965 is missing from both MDA and Risk
Data for CIK 1712189 is missing from both MDA and Risk
Data for CIK 1747009 is missing from both MDA and Risk
Data for CIK 1792580 is missing from both MDA and Risk
Data for CIK 1738827 is missing from both MDA and Risk
Data for CIK 1724009 is missing from both MDA and Risk
Data for CIK 1529463 is missing from both MDA and Risk
Data for CIK 1811074 is missing from both MDA and Risk
Data for CIK 1458891 is missing from both MDA and Risk
Data for CIK 1698514 is missing from both MDA and Risk
Data for CIK 1708301 is missing from both MDA and Risk
Data for CIK 1748773 is missing from both MDA and Risk
Data for CIK 1699136 is missing from both MDA and Risk
Data for CIK 1836875 is missing from both MDA and Risk
Data for CIK 1694028 is missing from both MDA and Risk
Data for CIK 1792849 is missing from both MDA and Risk
Data for C

### Remove companies that are not covering requirements from DF

In [211]:
list_of_ciks_to_remove = list(set(list(dict_mda_consecutives.keys()) + list(dict_risk_consecutives.keys())))
df_cleaned = df_cleaned.loc[~df_cleaned['cik'].isin(list_of_ciks_to_remove)]

In [159]:
fig = px.histogram(df_cleaned, x="len_text",color="section_type", marginal="rug",
                   hover_data=df_cleaned.columns, title="Processed companies / All filings from 2017 to 2022 by section type")
fig.show()

In [160]:
fig = px.histogram(df_cleaned.loc[df_cleaned['section_type'] == 'risk'], x="len_text", color="filing_type", marginal="rug",
                   hover_data=df_cleaned.columns, title="Processed companies / Risk sections from 2017 to 2022 by filing type")
fig.show()

In [3]:
# Number of risk sections per company and number of mda section per company

def count_section_availablity(df):
    dict_of_res = {}
    list_ciks = list(set(df['cik']))
    for cik in list_ciks:
        cik_df = df.loc[df['cik'] == cik]
        dict_of_res[cik] = {'risk': cik_df['section_type'].value_counts().get('risk'), 'mda': cik_df['section_type'].value_counts().get('mda')}
    return dict_of_res


In [4]:
dict_count_section = count_section_availablity(df_cleaned)
dict_count_section

{70145: {'risk': 20, 'mda': 20},
 874499: {'risk': 21, 'mda': 21},
 1537028: {'risk': 20, 'mda': 20},
 1524741: {'risk': 20, 'mda': 20},
 10254: {'risk': 19, 'mda': 19},
 1449488: {'risk': 20, 'mda': 20},
 1602065: {'risk': 20, 'mda': 20},
 313364: {'risk': 19, 'mda': 19},
 1038357: {'risk': 20, 'mda': 20},
 1171486: {'risk': 21, 'mda': 21},
 56868: {'risk': 19, 'mda': 19},
 1525287: {'risk': 20, 'mda': 20},
 1522727: {'risk': 20, 'mda': 20},
 1587246: {'risk': 21, 'mda': 21},
 1572910: {'risk': 20, 'mda': 20},
 1255474: {'risk': 20, 'mda': 20},
 1681459: {'risk': 20, 'mda': 20},
 77877: {'risk': 20, 'mda': 20},
 1006655: {'risk': 20, 'mda': 20},
 1126975: {'risk': 20, 'mda': 20},
 1084991: {'risk': 19, 'mda': 19},
 1693256: {'risk': 19, 'mda': 19},
 351817: {'risk': 20, 'mda': 20},
 890447: {'risk': 20, 'mda': 20},
 1282648: {'risk': 20, 'mda': 20},
 710752: {'risk': 20, 'mda': 20},
 1273441: {'risk': 21, 'mda': 21},
 893538: {'risk': 20, 'mda': 20},
 1549922: {'risk': 20, 'mda': 20},

## Save dataframe to reuse 

In [222]:
df_cleaned.to_csv('D:/PythonProjects/MasterThesisResearch/Notebooks/data/df_text_cleaned.csv', index=False)

## Load the cleaned dataframe

In [1]:
import pandas as pd

df_cleaned = pd.read_csv('D:/PythonProjects/MasterThesisResearch/Notebooks/data/df_text_cleaned.csv')

In [2]:
print(f"Number of companies in the corpus: {len(set(df_cleaned['cik']))}")

Number of companies in the corpus: 125


## Prepare samples for input based on company_type and filing_date

In [39]:
from datetime import datetime

def is_filing_on_time(df_filings_deadlines, filing_type, quarter, company_type, filing_date):
    filing_date_datetime = datetime.fromisoformat(filing_date)
    deadline_str = df_filings_deadlines.query(f'filing_type=="{filing_type}-{quarter}"')[f'{company_type}'][0]
    deadline_str = deadline_str.replace("year", str(filing_date_datetime.year))
    deadline_datetime = datetime.strptime(deadline_str, '%Y-%d-%m')
    print("Filing_date", filing_date_datetime)
    print("Deadline_date", deadline_datetime)
    if filing_date_datetime > deadline_datetime:
        return False
    return True
    

In [40]:
import pandas as pd
import json
import requests

list_of_ciks = [2178]

df_filing_deadlines = pd.read_csv("data/filing_deadlines.csv")

for cik in list_of_ciks:
    resp = json.loads(requests.get(f"http://localhost:8000/api/v1/company/{cik}").text)
    resp_stock_prices = json.loads(requests.get(f"http://localhost:8000/api/v1/stock_price/inflation/{cik}").text)
    if resp['code'] == 200:
        curr_company_list = resp['data']
    else:
        print(resp['message'])
    if resp_stock_prices['code'] == 200:
        curr_stock_prices_list = resp_stock_prices['data']
    else:
        print(resp_stock_prices['message'])
    
    for company_year_dict in curr_company_list:
        for quarter in company_year_dict['quarters']:
            curr_input = {}
            for metadata in quarter['metadata']:
                res = is_filing_on_time(df_filing_deadlines, metadata['type'], quarter['q'], metadata['company_type'], metadata['filing_date'])
                break
            break
        break
        

Filing_date 2017-03-31 00:00:00
Deadline_date 2017-03-16 00:00:00


In [41]:
res

False

In [20]:
from datetime import datetime

datetime.fromisoformat(metadata['filing_date']).year

2017

In [10]:
df_filing_deadlines.query('filing_type=="10K-1"')['accelerated_filer'][0]

'year-16-03'